In [5]:
# Import libraries
from utils import *

In [6]:
# Load data (please note to change file location below)
df = pd.read_csv('~/weather_data_bucharest_2000_2023.csv', encoding = "ISO-8859-1")

In [7]:
# Split data into independent and dependent variables. The first column should be the dependent variable y.
y = df.iloc[:, 0] # The Y-axis variable
x = df.iloc[:, 1:] #.drop(columns='temp', axis=1)

In [8]:
# Split the data into 3 samples: train, cross validation (cv) and test
x_train, x_, y_train, y_ = train_test_split(x, y, train_size=.6, shuffle=False)
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=.5, shuffle=False)